# Molecular Dynamics 4 

Ok, now that we have multiple atoms, lets have them interact with each other.  
  
So lets restart all of our previous turtle things to get window set up.

In [1]:
import turtle
import random
import math

When using random numbers, it is important to start a seed to insure that you actually are getting random numbers.

In [2]:
random.seed()

Now lets get our back drop set up.

In [3]:
window = turtle.Screen()
window.title('Molecular Dynamics 4')
window.clear()

Lets see how big our window is, which will tell us where the boundaries are.

In [4]:
# We will want to store these numbers in a variable
height = window.window_height()
width = window.window_width()

Lets check and see what we have here.

In [5]:
print height
print width

810
960


Now we have our window, lets make a turtles to draw our atoms. We are going to make a list of atoms and then populate it.

In [6]:
# Number of atoms we want
num_atoms = 5

# This initializes an empty list
atoms = []

# Now use a loop to initialize a new atom
# and do it num_atoms times.
for i in range(num_atoms):
    atoms.append(turtle.Turtle())

We should now have two atoms stored in atom_list. We can check various aspects of this new list using print.

In [7]:
print len(atoms)

5


In [8]:
print atoms

[<turtle.Turtle object at 0x10c7eb110>, <turtle.Turtle object at 0x10706bfd0>, <turtle.Turtle object at 0x10c7eb210>, <turtle.Turtle object at 0x10c7eb2d0>, <turtle.Turtle object at 0x10c7eb390>]


Everything looks good. Now lets draw our atoms as spheres with a fixed radius and random color and place them at random locations.

In [9]:
# Variables to hold things we want to be constant
atom_radius = 20

# Scaling factor here is so we don't get our 
# initial positions stuck on the edge.
scaling_factor = 0.8

# We need to loop over each atom.
for i in range(num_atoms):
   
    # Draw the atom in the proper shape
    atoms[i].shape('circle')
    atoms[i].shapesize(atom_radius/10.0)
    atoms[i].color((random.random(),random.random(),random.random()))
    atoms[i].penup()
    atoms[i].goto(random.uniform(-1,1)*width/2.0 * scaling_factor, random.uniform(-1,1)*height/2.0 * scaling_factor)
  
    # Turtles can be very slow. This is a semi-fix to tell
    # turtles not to update the screen with every change,
    # but rather wait till a set of updates are done and 
    # then update the screen.
    atoms[i].tracer(0,0)
    turtle.update()

Alright, now lets make a vector with random velocities for each atom. Our velocity vector will have one line per atom, and each line will have a pair of floating point numbers that represent (velocity_x, velocity_y).

In [10]:
# Max velocity we want
max_velocity = 20.0

# Initailize an empty list
vel = []

# Now loop over the number of atoms
for i in range(num_atoms):
    vel.append([random.uniform(-1,1)*max_velocity, random.uniform(-1,1)*max_velocity])

Lets check to make sure this worked.

In [11]:
print len(vel)

5


In [12]:
print vel

[[-7.749428634892288, -18.561081226501077], [-10.388630516661177, -16.713525356238726], [-3.3035690249581817, 1.5519834405426414], [14.672282376134387, -10.775313665178516], [8.45345707475946, 16.402035238931376]]


Now how would we access only 1 element? We use square brackets []!

In [13]:
print vel[0]

[-7.749428634892288, -18.561081226501077]


Now how would we access the x_velocity component here? We'd use another set of square brackets!

In [14]:
print vel[0][0]

-7.74942863489


Now we need to write a function that will handle the interaction of any two atoms.  
  
First we will need, at some point, to calculate distances between two atoms, so lets write a function that handles that.

In [15]:
# Lets name our function
def distance(atom1, atom2):
    
    # Inputs are the atoms, which contain
    # (x,y) coordinate pairs.
    # Distance is sqrt((x1-x2)^2 + (y1-y2)^2)
    
    # Lets get the coordinates of atom1
    (x1,y1) = atom1.pos()
    
    # Lets get the coordinates of atom2
    (x2,y2) = atom2.pos()

    # Lets calculate the distance
    d = math.sqrt((x1-x2)**2 + (y1-y2)**2)
    
    # And lets return our distance
    return d
    

Now lets write a function to determine if our atoms will collide, and will handle what to do when they collide.

In [16]:
# Lets name our function
def bounce_check(a_list, v_list):
  
    # This will help us not double count
    i = 1

    # We need to check for every atom
    for atom1 in a_list:
        
        # This will help us not double count
        j = i
        
        # And compute the distance to every other atom
        for atom2 in a_list[i:]:
            
            # We will need the distance, so store it
            d = distance(atom1,atom2)
            
            # Check if a bounce should occur
            if d < 2.0*atom_radius:
                print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                # We need positions of atoms
                (x1,y1) = atom1.pos()
                (x2,y2) = atom2.pos()
                
                # A bounce should occur: fix x_velocity
                # First, calculate v1.r
                v1R = v_list[i-1][0]*(x1-x2) + v_list[i-1][1]*(y1-y2)
                v2R = v_list[j][0]*(x1-x2) + v_list[j][1]*(y1-y2)
                
                # Update new velocities
                v_list[i-1][0] = v_list[i-1][0] - 2.0*v1R/d**2 * (x1-x2)
                v_list[i-1][1] = v_list[i-1][1] - 2.0*v1R/d**2 * (y1-y2)
                v_list[j][0] = v_list[j][0] - 2.0*v2R/d**2 * (x1-x2)
                v_list[j][1] = v_list[j][1] - 2.0*v2R/d**2 * (y1-y2)

            # Update second counter
            j = j + 1
            
        # Update counter
        i = i + 1

In [17]:
# Lets name our function
def forces_check(atom_list, v_list, dt):

    # Our spring constant
    k = 0.05

    # This will help us not double count
    i = 1

    # We need to check for every atom
    for atom1 in atom_list:
        
        # This will help us not double count
        j = i
        
        # And compute the distance to every other atom
        for atom2 in atom_list[i:]:
            
            # We will need the distance, so store it
            d = distance(atom1,atom2)
            
            # Check if a bounce should occur
            if d < 2.0*atom_radius:
                print "BOUNCE FOUND: Atom %d hit Atom %d" % (i-1,j)
                # We need positions of atoms
                (x1,y1) = atom1.pos()
                (x2,y2) = atom2.pos()
                
                # A bounce should occur
                # Update new velocities
                v_list[i-1][0] = v_list[i-1][0] + dt/2.0*k*d*(x1-x2)
                v_list[i-1][1] = v_list[i-1][1] + dt/2.0*k*d*(y1-y2)
                v_list[j][0] = v_list[j][0] + dt/2.0*k*d*(x2-x1)
                v_list[j][1] = v_list[j][1] + dt/2.0*k*d*(y2-y1)

            # Update second counter
            j = j + 1
            
        # Update counter
        i = i + 1

In [18]:
def forces(coords):
    ''' Compute the forces, the potential energy '''
    
    epsilon = 20.0   # LJ parameter
    sigma = 20.0    # LJ parameter
    
    f = [(0.0,0.0)]*len(coords)
    pe = 0.0
    
    # V(ri-rj) = epsilon*((sigma/r)^12 - 2*(sigma/r)^6)
    # dV/dxi = -12*epsilon*((sigma/r)^14 - (sigma/r)^8)*(xi-xj)/sigma**2
    # F[i][x] = -dV/dxi
    
    fac = epsilon*12.0/sigma**2
    
    for i in range(len(coords)):
        
        for j in range(i+1,len(coords)):
            xi,yi = coords[i].pos()
            xj,yj = coords[j].pos()
            dx,dy = (xi-xj),(yi-yj)
            rsq = (dx**2+dy**2)/sigma**2
        
            # Error if true: print to help debug why
            if rsq == 0.0:
                print "%.8f %.8f %.8f %.8f %d %d" % (xi,yi,xj,yj,i,j)
            
            if rsq < 15:
                vij = epsilon*(1.0/rsq**6 - 2.0/rsq**3)
                df = fac*(1.0/rsq**7 - 1.0/rsq**4)
                dfx,dfy = df*dx,df*dy
                fxi,fyi = f[i]
                fxj,fyj = f[j]
                f[i] = fxi+dfx,fyi+dfy
                f[j] = fxj-dfx,fyj-dfy
                pe += vij

    return f, pe

Now to start moving our atoms.

In [19]:
# The amount of time each iteration moves us forward
dt = 0.5

# Max number of steps we want to take
max_steps = 1

# Print headers for output
print " "
print "   time         ke            pe             e      "
print " -------    -----------   ------------  ------------"  

for step in range(max_steps):

    # Calculate forces
    f, pe = forces(atoms)
    
    # For each atom
    for j in range(num_atoms):
    
        # Get the current position, velocity, and force of the atom
        vx,vy = vel[j]    
        Fx,Fy = f[j]  
        vx += Fx*dt*0.5
        vy += Fy*dt*0.5
        x,y = atoms[i].pos()
    
        # Check if moving left or right will put our atom beyond the wall
        if abs(x + dt * vel[j][0]) >= width/2.0 - atom_radius:
        
            # We have moved too far right or left, so flip the x_vel
            vel[j][0] = -vel[j][0]           
    
        # Check if moving up or down will put our atom beyond the wall
        if abs(y + dt * vel[j][1]) >= height/2.0 - atom_radius:
        
            # We have moved too far up or down, so flip the y_vel
            vel[j][1] = -vel[j][1]
        
        # We won't move out of the box, so update the new position
        atoms[j].goto(x + dt*vel[j][0], y + dt*vel[j][1])
        
        
    # make the forces at time t+dt
    f, potential_energy = forces(atoms)
            
    # finish update of v to time t+dt
    kinetic_energy = 0.0
    for i in range(num_atoms):
        vx,vy = vel[i]
        Fx,Fy = f[i]
        vx += Fx*dt*0.5
        vy += Fy*dt*0.5
        vel[i] = [vx, vy]
        kinetic_energy += 0.5*(vx*vx+vy*vy)
        
        
    # Tell turtles we are done updating and to redraw
    turtle.update()
    
    # Output energies to make sure its working
    energy = kinetic_energy + potential_energy
    print "%8.2f   %12.5f   %12.5f  %12.5f  " % (step*dt, kinetic_energy, potential_energy, energy) 
    

 
   time         ke            pe             e      
 -------    -----------   ------------  ------------
    0.00   249955320845993540649576562688.00000   268444448475951.62500  249955320845993822124553273344.00000  


It works!

Always clean up.

In [20]:
window.bye()